<a href="https://colab.research.google.com/github/Pengyu-gis/Environmental_Hazard_Course_Final/blob/main/GEE_COUNTY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install earthengine-api

In [1]:
# Import libraries
import ee
import geemap
import folium

In [ ]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()

In [10]:
ee.Initialize(project='ee-pengyuchen2002')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [18]:
# Load the Feature Collection for U.S. counties
counties = ee.FeatureCollection('TIGER/2018/Counties')

# Define Chester County, PA
chester_pa = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Chester'),
    ee.Filter.eq('STATEFP', '42')  # Pennsylvania FIPS code
))

# Get the geometry of Chester County
roi_chester = chester_pa.geometry()


In [19]:
# Define the time range
start_date = '2022-06-01'
end_date = '2022-08-31'  # Summer months

# Load Sentinel-2 surface reflectance data
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi_chester) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Function to mask clouds and scale the data
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Apply the cloud masking function
s2_clean = s2_collection.map(mask_s2_clouds)

# Create a median composite image
composite_chester = s2_clean.median().clip(roi_chester)


In [20]:
# Load NLCD land cover data
nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019') \
    .select('landcover') \
    .clip(roi_chester)

# Define the bands to use
bands = ['B2', 'B3', 'B4', 'B8']  # Blue, Green, Red, NIR

# Sample the input imagery and NLCD data
sample_points = nlcd.stratifiedSample(
    numPoints=2000,
    classBand='landcover',
    region=roi_chester,
    scale=30,
    geometries=True,
    seed=0
)

# Extract the pixel values for each sample point
training_data = composite_chester.select(bands).sampleRegions(
    collection=sample_points,
    properties=['landcover'],
    scale=30
)


In [21]:
# Initialize the classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50)

# Train the classifier
trained_classifier = classifier.train(
    features=training_data,
    classProperty='landcover',
    inputProperties=bands
)


In [22]:
# Classify the composite image
classified_image = composite_chester.select(bands).classify(trained_classifier)


In [23]:
# Define land cover class names and colors (NLCD palette)
nlcd_palette = [
    '000000',  # Background (0)
    '466b9f',  # Open Water (11)
    'd1def8',  # Perennial Ice/Snow (12)
    'dec5c5',  # Developed, Open Space (21)
    'd99282',  # Developed, Low Intensity (22)
    'eb0000',  # Developed, Medium Intensity (23)
    'ab0000',  # Developed High Intensity (24)
    'b3ac9f',  # Barren Land (31)
    '68ab5f',  # Deciduous Forest (41)
    '1c5f2c',  # Evergreen Forest (42)
    'b5c58f',  # Mixed Forest (43)
    'af963c',  # Dwarf Scrub (51)
    'ccb879',  # Shrub/Scrub (52)
    'dfdfc2',  # Grassland/Herbaceous (71)
    'd1d182',  # Pasture/Hay (81)
    'a3cc51',  # Cultivated Crops (82)
    '82ba9e',  # Woody Wetlands (90)
    'a8daba',  # Emergent Herbaceous Wetlands (95)
]

# Map visualization parameters
vis_params = {
    'min': 0,
    'max': 95,
    'palette': nlcd_palette
}

# Initialize the map
Map = geemap.Map(center=[39.9, -75.7], zoom=10)

# Add the composite image
Map.addLayer(
    composite_chester.select(['B4', 'B3', 'B2']),
    {'min': 0, 'max': 0.3},
    'Sentinel-2 RGB'
)

# Add the classified image
Map.addLayer(
    classified_image,
    vis_params,
    'Chester County Classification'
)

# Display the map
Map


Map(center=[39.9, -75.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [25]:
# Define the class labels and their corresponding colors
legend_dict = {
    'Open Water': '466b9f',
    'Developed, Open Space': 'd1def8',
    'Developed, Low Intensity': 'dec5c5',
    'Developed, Medium Intensity': 'd99282',
    'Developed High Intensity': 'eb0000',
    'Barren Land': 'ab0000',
    'Deciduous Forest': 'b3ac9f',
    'Evergreen Forest': '68ab5f',
    'Mixed Forest': '1c5f2c',
    'Shrub/Scrub': 'b5c58f',
    'Grassland/Herbaceous': 'af963c',
    'Pasture/Hay': 'ccb879',
    'Cultivated Crops': 'dfdfc2',
    'Woody Wetlands': 'd1d182',
    'Emergent Herbaceous Wetlands': 'a3cc51',
}

# Visualization parameters
vis_params = {
    'min': 0,
    'max': 95,
    'palette': nlcd_palette
}

# Initialize the map
Map = geemap.Map(center=[39.9, -75.7], zoom=10)

# Add the composite image
Map.addLayer(
    composite_chester.select(['B4', 'B3', 'B2']),
    {'min': 0, 'max': 0.3},
    'Sentinel-2 RGB'
)

# Add the classified image
Map.addLayer(
    classified_image,
    vis_params,
    'Chester County Classification'
)

# Add the legend
Map.add_legend(legend_title='Land Cover Classification', legend_dict=legend_dict)


# Display the map
Map

Map(center=[39.9, -75.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

# howard

In [28]:
# Define Howard County, MD
howard_md = counties.filter(ee.Filter.And(
    ee.Filter.eq('NAME', 'Howard'),
    ee.Filter.eq('STATEFP', '24')  # Maryland FIPS code
))

# Get the geometry of Howard County
roi_howard = howard_md.geometry()


In [29]:
# Load Sentinel-2 surface reflectance data for Howard County
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi_howard) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Apply the cloud masking function
s2_clean = s2_collection.map(mask_s2_clouds)

# Create a median composite image
composite_howard = s2_clean.median().clip(roi_howard)


In [30]:
# Load NLCD land cover data
nlcd = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019') \
    .select('landcover') \
    .clip(roi_howard)

# Sample the input imagery and NLCD data
sample_points = nlcd.stratifiedSample(
    numPoints=2000,
    classBand='landcover',
    region=roi_howard,
    scale=30,
    geometries=True,
    seed=0
)

# Extract the pixel values for each sample point
training_data = composite_howard.select(bands).sampleRegions(
    collection=sample_points,
    properties=['landcover'],
    scale=30
)


In [31]:
# Initialize the classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=50)

# Train the classifier
trained_classifier = classifier.train(
    features=training_data,
    classProperty='landcover',
    inputProperties=bands
)


In [32]:
# Classify the composite image
classified_image = composite_howard.select(bands).classify(trained_classifier)


In [35]:
# Initialize the map
Map = geemap.Map(center=[39.25, -76.93], zoom=10)

# Add the composite image
Map.addLayer(
    composite_howard.select(['B4', 'B3', 'B2']),
    {'min': 0, 'max': 0.3},
    'Sentinel-2 RGB'
)

# Add the classified image
Map.addLayer(
    classified_image,
    vis_params,
    'Howard County Classification'
)

# Define the class labels and their corresponding colors
legend_dict = {
    'Open Water': '466b9f',
    'Developed, Open Space': 'd1def8',
    'Developed, Low Intensity': 'dec5c5',
    'Developed, Medium Intensity': 'd99282',
    'Developed High Intensity': 'eb0000',
    'Barren Land': 'ab0000',
    'Deciduous Forest': 'b3ac9f',
    'Evergreen Forest': '68ab5f',
    'Mixed Forest': '1c5f2c',
    'Shrub/Scrub': 'b5c58f',
    'Grassland/Herbaceous': 'af963c',
    'Pasture/Hay': 'ccb879',
    'Cultivated Crops': 'dfdfc2',
    'Woody Wetlands': 'd1d182',
    'Emergent Herbaceous Wetlands': 'a3cc51',
}

# Add the legend to the map
Map.add_legend(legend_title='Land Cover Classification', legend_dict=legend_dict)

# Display the map
Map



Map(center=[39.25, -76.93], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…